## 12 - Evidência

### 12.1 - Interpretando as pontuações do SAT

- SAT: Um teste padronizado destinado a medir a preparação para o trabalho de nível universitário em matemática;

Se Alice obteve 780 e Bob obteve um 740 (de um possível 800), convém saber se essa diferença é uma evidência de que Alice está melhor preparada que Bob e qual é a força dessa evidência.

- Quão forte é a evidência de que Alice está melhor preparada que Bob?
- Finjo, temporariamente, que todas as perguntas do SAT são igualmente difíceis.

- `p_correct`: Probabilidade de responder corretamente a qualquer pergunta;

### 12.2 A escala

- Entender o processo de pontuação e dimensionamento do SAT;
    - Pontuação bruta com base no número de perguntas corretas e incorretas;
    - A pontuação é escalada no intervalo de 200 a 800;

    - A pontuação bruta é o número de perguntas respondidas corretamente, menos uma penalidade de 1/4 de ponto para cada pergunta respondida incorretamente.


### 12.3 - A priori

- Podemos usar a distribuição de pontuações brutas para modelar a distribuição a priori de `p_correct`.

In [2]:
class Exam(object):

    def __init__(self):
        self.scale = ReadScale()
        scores = ReadRanks()
        score_pmf = thinkbayes.MakePmfFromDict(dict(scores))
        self.raw = self.ReverseScale(score_pmf)
        self.max_score = max(self.raw.Values())
        self.prior = DivideValues(self.raw, self.max_score)

![](p_corret.png)

- Distribuição a priori de `p_correct` para os participantes do teste SAT.
- Essa distribuição é aproximadamente gaussiana, mas é comprimida nos extremos
- o SAT tem **mais poder de diferenciar** entre os participantes do teste **dentro de dois desvios padrão da média** e menos poder fora desse intervalo.

In [7]:
import thinkbayes
class Sat(thinkbayes.Suite):

    def __init__(self, exam, score):
        thinkbayes.Suite.__init__(self)

        self.exam = exam
        self.score = score

        # start with the prior distribution
        for p_correct, prob in exam.prior.Items():
            self.Set(p_correct, prob)

        # update based on an exam score
        self.Update(score)
        
    def Likelihood(self, data, hypo):
        p_correct = hypo
        score = data

        k = self.exam.Reverse(score)
        n = self.exam.max_score
        like = thinkbayes.EvalBinomialPmf(k, n, p_correct)
        return like

- `hypo` é um valor hipotético de `p_correct`
- `data` é uma pontuação escalada.

- Pontuação bruta como o número de respostas corretas, ignorando a penalidade por respostas erradas;
- A verossimilhança é dada pela distribuição binomial, que calcula a probabilidade de k respostas corretas de n perguntas.

### 12.4 - Posteriori

![](posteriori_12.png)

- Distribuições posteriores de `p_correct` para Alice e Bob.

- Quão forte é a evidência de que Alice está melhor preparada que Bob?

Para formular a pergunta em termos de teste de hipótese bayesiana, defino duas hipóteses:

- A: `p_correct` mais alto para Alice do que para Bob.
- B: `p_correct` mais alto para Bob do que para Alice.

Para calcular a verossimilhança de **A** , podemos enumerar todos os pares de valores das distribuições posteriores e somar a probabilidade total dos casos em que `p_correct` é maior para Alice do que para Bob.

In [8]:
class TopLevel(thinkbayes.Suite):

    def Update(self, data):
        a_sat, b_sat = data

        a_like = thinkbayes.PmfProbGreater(a_sat, b_sat)
        b_like = thinkbayes.PmfProbLess(a_sat, b_sat)
        c_like = thinkbayes.PmfProbEqual(a_sat, b_sat)

        a_like += c_like / 2
        b_like += c_like / 2

        self.Mult('A', a_like)
        self.Mult('B', b_like)

        self.Normalize()

- **a_like** é a probabilidade total que `p_correct` é maior para Alice; 
- **b_like** é a probabilidade de que seja maior para Bob;
- **c_like** é a probabilidade de serem "iguais";

Se usarmos mais valores, c_like será menor e, no extremo, se `p_correct` for contínuo, c_like será zero;
- c_like, tipo de erro de arredondamento e divido igualmente entre a_like e b_like.

- A verossimilhança de A é 0,79 e a verossimilhança de B é 0,21. 
- A razão de verossimilhança (ou fator Bayes) é de 3.8.
    - O que significa que a evidência de que Alice é melhor que Bob em responder perguntas do SAT. 

A probabilidade de A é de 79%, o que significa que ainda há 21% de chance de Bob estar realmente melhor preparado.

### 12.5 - Um modelo melhor

- Qual é o tamanho do erro de modelagem?
    - Se for Pequeno, concluímos que o primeiro modelo (todas as perguntas são igualmente difícei) é bom o suficiente.
    - Se for grande, precisamos de um modelo melhor.
   

- Suponha que cada participante do teste tenha algum grau de eficácia (`efficacy`), que mede sua capacidade de responder às perguntas do SAT.
- Suponha que cada pergunta tenha algum nível de dificuldade (`difficulty`).
- Por fim, suponha que a chance de um participante do teste responder a uma pergunta corretamente esteja relacionada à eficácia (`efficacy`) e à dificuldade (`difficulty`);

In [1]:
def ProbCorrect(efficacy, difficulty, a=1):
    return 1 / (1 + math.exp(-a * (efficacy - difficulty)))

- Versão simplificada da curva usada na teoria de resposta ao item;
- Eficácia e a dificuldade são consideradas na mesma escala;

- A probabilidade de acertar uma pergunta depende da diferença entre eficácia e a dificuldade.
- Quando eficácia e dificuldade são iguais, a probabilidade de acertar a pergunta é de 50%;
- À medida que a eficácia aumenta (ou à medida que a dificuldade diminui), a probabilidade se aproxima de 100%;
- À medida que a eficácia diminui (ou à medida que a dificuldade aumenta), a probabilidade se aproxima de 0%.

In [2]:
def PmfCorrect(efficacy, difficulties):
    pmf0 = thinkbayes.Pmf([0])

    ps = [ProbCorrect(efficacy, diff) for diff in difficulties]
    pmfs = [BinaryPmf(p) for p in ps]
    dist = sum(pmfs, pmf0)
    return dist

- `difficulties` é uma lista de dificuldades, uma para cada pergunta. 
- `ps` é uma lista de probabilidades;
- `pmfs` é uma lista de objetos Pmf de dois valores

- Para um grupo de pessoas com eficácia diferente, a distribuição resultante das pontuações brutas é uma mistura

In [3]:
# class Exam:

    def MakeRawScoreDist(self, efficacies):
        pmfs = thinkbayes.Pmf()
        for efficacy, prob in efficacies.Items():
            scores = PmfCorrect(efficacy, self.difficulties)
            pmfs.Set(scores, prob)

        mix = thinkbayes.MakeMixture(pmfs)
        return mix

### 12.6 - Calibração

- Recebemos a distribuição de pontuações brutas e queremos inferir a distribuição de dificuldades.

![](dist_data_model.png)

Distribuição real de pontuações brutas e um modelo para ajustá-lo.

- Distribuição da dificuldade seja uniforme com os parâmetros center and width;

In [5]:
def MakeDifficulties(center, width, n):
    low, high = center-width, center+width
    return numpy.linspace(low, high, n)

`center = -0.05` e `width = 1.8` produzem uma distribuição de pontuações brutas semelhantes aos dados reais;

- Intervalo da distribuição de dificuldade é de aproximadamente -1,85 a 1,75;
- Eficácia é gaussiana com média 0 e desvio padrão 1.5;

![](table_cap12.png)

### 12.7 - Distribuição posteriori da eficácia

![](post_eficacia.png)

Distribuições posteriores de eficácia para Alice e Bob.

In [8]:
class Sat2(thinkbayes.Suite):

    def __init__(self, exam, score):
        self.exam = exam
        self.score = score

        # start with the Gaussian prior
        efficacies = thinkbayes.MakeGaussianPmf(0, 1.5, 3)
        thinkbayes.Suite.__init__(self, efficacies)

        # update based on an exam score
        self.Update(score)
        
    def Likelihood(self, data, hypo):
        efficacy = hypo
        score = data
        raw = self.exam.Reverse(score)

        pmf = self.exam.PmfCorrect(efficacy)
        like = pmf.Prob(raw)
        return like

- A razão de verossimilhança é 3.4, um pouco menor do que a obtida no modelo simples (3.8). 
- A priori é de que A e B são igualmente prováveis, A tem uma probabilidade posteriori de 77%, deixando 23% de chance de que a eficácia de Bob seja maior;

### 12.8 - Distribuição preditiva

- Estimativas para a eficácia de Alice e Bob;
- Se Alice e Bob fizerem o teste de matemática novamente, qual é a chance de Alice fazer melhor novamente?

- Usaremos a distribuição posteriori da eficácia para gerar uma distribuição preditiva da pontuação bruta para cada participante do teste.
- Compararemos as duas distribuições preditivas para calcular a probabilidade de Alice obter uma pontuação mais alta novamente.

- A probabilidade de Alice é de 63%;
- A de Bob é 37% de chance de se sair tão bem ou melhor;

### 12.9 Discussão